In [ ]:
from itertools import chain

from transformers import AutoConfig, IntervalStrategy, TrainingArguments

from utils.base_directory import base_directory
from utils.model_utils import finetune_model
from utils.dataset_utils import get_leaning_datasets_for_leave_one_out_benchmark, get_leave_one_out_datasets
import optuna

In [ ]:
TRAINING_POLITICAL_LEANING = True
GET_DATASETS = lambda: chain(
    get_leaning_datasets_for_leave_one_out_benchmark()
)
TEST_DATASET_SAMPLE_FRACTION = 0.15
EVAL_DATASET_SAMPLE_SIZE = 1_000
TRAIN_DATASET_SAMPLE_SIZE = 10_000

BALANCE_CENTER_LEANING_CLASS = False
CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIERS = {
    "article_bias_prediction": 5,
    "bignewsbln": 10,  # There is not enough center leaning examples to balance this out.
    "commoncrawl_news_articles": 5.5,
    "gpt4_political_bias": 3.25,
    "gpt4_political_ideologies": 3,
    "qbias": 3.25,
}
CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIER_DEFAULT = 2.6

train_datasets, eval_datasets = get_leave_one_out_datasets(
    list(GET_DATASETS()),
    TRAINING_POLITICAL_LEANING,
    TEST_DATASET_SAMPLE_FRACTION,
    EVAL_DATASET_SAMPLE_SIZE,
    TRAIN_DATASET_SAMPLE_SIZE,
    BALANCE_CENTER_LEANING_CLASS,
    CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIERS,
    CENTER_LEANING_CLASS_TRAIN_SIZE_MULTIPLIER_DEFAULT,
)

In [ ]:
MODEL_NAME = "launch/POLITICS"
LEFT_OUT_DATASET_INDEX = 0
TRIALS_COUNT = 100

def objective(trial):
    attention_dropout = trial.suggest_float("attention_dropout", 0.1, 0.3)
    hidden_dropout = trial.suggest_float("hidden_dropout", 0.1, 0.5)
    classifier_dropout = trial.suggest_float("classifier_dropout", 0.0, 0.5)

    learning_rate = trial.suggest_float(
        "learning_rate", 1e-5, 7e-5, log=True
    )
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.05, 0.25)
    batch_size = trial.suggest_categorical(
        "batch_size", [8, 16, 32, 48, 64]
    )
    weight_decay = trial.suggest_float(
        "weight_decay", 0.0001, 0.1, log=True
    )

    print(f"attention dropout: {attention_dropout}")
    print(f"hidden dropout: {hidden_dropout}")
    print(f"classifier dropout: {classifier_dropout}")
    print(f"learning rate: {learning_rate}")
    print(f"warmup ratio: {warmup_ratio}")
    print(f"batch size: {batch_size}")
    print(f"weight decay: {weight_decay}")

    model_config = AutoConfig.from_pretrained(
        MODEL_NAME,
        attention_probs_dropout_prob=attention_dropout,
        hidden_dropout_prob=hidden_dropout,
        classifier_dropout=classifier_dropout,
    )

    trial_id = f"{trial.number}_{attention_dropout}_{hidden_dropout}_{classifier_dropout}_{learning_rate}_{warmup_ratio}_{batch_size}_{weight_decay}"

    steps = 19_200 / batch_size

    training_arguments = TrainingArguments(
        num_train_epochs=5,
        learning_rate=learning_rate,
        warmup_ratio=warmup_ratio,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=1,
        weight_decay=weight_decay,
        eval_strategy=IntervalStrategy.STEPS,
        save_strategy=IntervalStrategy.STEPS,
        save_steps=steps,
        save_total_limit=3,  # https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments.save_total_limit
        load_best_model_at_end=True,
        metric_for_best_model="f1",  # Defaults to "loss".
        greater_is_better=True,
        output_dir=base_directory / "models" / "hyperparameter_search" / trial_id,
        logging_steps=steps,
        logging_dir=f"./logs_{trial_id}",
        report_to="tensorboard",
        seed=37,
        data_seed=37,
    )

    trainer = finetune_model(
        train_datasets[LEFT_OUT_DATASET_INDEX],
        eval_datasets[LEFT_OUT_DATASET_INDEX],
        MODEL_NAME,
        training_arguments,
        model_config,
    )
    metrics = trainer.evaluate()

    del trainer

    return metrics["eval_f1"]

study = optuna.create_study(
    direction="maximize"
)
study.optimize(objective, n_trials=TRIALS_COUNT)

study

In [ ]:
study.best_trial

In [ ]:
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

In [ ]:
study.best_params